In [1]:
import numpy as np
import os

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
os.makedirs("lfw")

In [4]:
!unzip "/content/drive/MyDrive/cropped (2).zip" -d "lfw"

Streaming output truncated to the last 5000 lines.
  inflating: lfw/content/cropped/Lesley_McCulloch/Lesley_McCulloch_0002.jpg  
  inflating: lfw/content/cropped/Lesley_McCulloch/Lesley_McCulloch_0003.jpg  
   creating: lfw/content/cropped/Anne_McLellan/
  inflating: lfw/content/cropped/Anne_McLellan/Anne_McLellan_0001.jpg  
  inflating: lfw/content/cropped/Anne_McLellan/Anne_McLellan_0003.jpg  
  inflating: lfw/content/cropped/Anne_McLellan/Anne_McLellan_0002.jpg  
   creating: lfw/content/cropped/Harry_Kalas/
  inflating: lfw/content/cropped/Harry_Kalas/Harry_Kalas_0002.jpg  
  inflating: lfw/content/cropped/Harry_Kalas/Harry_Kalas_0001.jpg  
   creating: lfw/content/cropped/Hee-Won_Han/
  inflating: lfw/content/cropped/Hee-Won_Han/Hee-Won_Han_0002.jpg  
  inflating: lfw/content/cropped/Hee-Won_Han/Hee-Won_Han_0001.jpg  
  inflating: lfw/content/cropped/Hee-Won_Han/Hee-Won_Han_0003.jpg  
   creating: lfw/content/cropped/Craig_Burley/
  inflating: lfw/content/cropped/Craig_Burley/Crai

In [6]:
import os
import numpy as np
import cv2

In [35]:
train_images= []
train_labels= []
for folder in os.listdir("/content/lfw/content/cropped/"):
  for imgs in os.listdir("/content/lfw/content/cropped/"+folder):
    img= cv2.imread("/content/lfw/content/cropped/"+folder+"/"+imgs,0)
    img= cv2.resize(img,(64,64))
    img= img.flatten()
    train_images.append(img)
    train_labels.append(folder)
train_images= np.array(train_images)
train_labels= np.array(train_labels)

In [ ]:
#train_images= train_images[0:12000]
#train_labels= train_labels[0:12000]

In [36]:
train_images.shape, train_labels.shape

((13232, 4096), (13232,))

In [ ]:
!unrar x "/content/drive/MyDrive/cropped_face.rar"

In [37]:
Y=[]
X=[]
for files in os.listdir("cropped_face/image1/0"):
  j=[]
  image1=cv2.imread(os.path.join("cropped_face/image1/0",files),0)
  image1=  cv2.resize(image1,(64,64))
  image2=cv2.imread(os.path.join("cropped_face/image2/0",files),0)
  image2=cv2.resize(image2,(64,64))
  image1=image1.flatten()
  image2=image2.flatten()
  j.append(image1)
  j.append(image2)
  j= np.array(j)
  X.append(j)
  Y.append(0)
for files in os.listdir("cropped_face/image1/1"):
  j=[]
  image1=cv2.imread(os.path.join("cropped_face/image1/1",files),0)
  image1=cv2.resize(image1,(64,64))
  image2=cv2.imread(os.path.join("cropped_face/image2/1",files),0)
  image2=cv2.resize(image2,(64,64))
  image1=image1.flatten()
  image2=image2.flatten()
  j.append(image1)
  j.append(image2)
  j= np.array(j)
  X.append(j)
  Y.append(1)

Eigen Face

In [10]:
from sklearn.decomposition import PCA
pca = PCA(n_components = 200)
train_images = pca.fit_transform(train_images)

In [17]:
outputs=[]
for x in range(0,len(X)):
  c = pca.transform(X[x])
  a= c[0]
  b= c[1]
  dist = np.linalg.norm(a - b)
  outputs.append(dist)

In [18]:
outputs= np.array(outputs)
Y= np.array(Y)

In [19]:
# Training the Kernel SVM model on the Training set
from sklearn.svm import SVC
classifier = SVC(kernel = 'rbf', random_state = 0)
from sklearn.model_selection import cross_val_score
accuracies = cross_val_score(estimator = classifier, X = outputs.reshape(-1,1), y = Y, cv = 10)
print("Accuracy: {:.2f} %".format(accuracies.mean()*100))
print("Standard Deviation: {:.2f} %".format(accuracies.std()*100))

Accuracy: 64.62 %
Standard Deviation: 1.79 %


In [25]:
from sklearn import metrics
from sklearn.model_selection import GridSearchCV, KFold, StratifiedKFold
from sklearn.metrics import classification_report, accuracy_score, make_scorer
originalclass = []
predictedclass = []
def classification_report_with_accuracy_score(y_true, y_pred):
    originalclass.extend(y_true)
    predictedclass.extend(y_pred)
    return accuracy_score(y_true, y_pred) # return accuracy score

inner_cv = StratifiedKFold(n_splits=10)
outer_cv = StratifiedKFold(n_splits=10)

# Nested CV with parameter optimization
nested_score = cross_val_score(classifier, X=outputs.reshape(-1,1), y=Y, cv=outer_cv, scoring=make_scorer(classification_report_with_accuracy_score))

# Average values in classification report for all folds in a K-fold Cross-validation  
print(classification_report(originalclass, predictedclass)) 

              precision    recall  f1-score   support

           0       0.64      0.66      0.65      3000
           1       0.65      0.63      0.64      3000

    accuracy                           0.65      6000
   macro avg       0.65      0.65      0.65      6000
weighted avg       0.65      0.65      0.65      6000



BAYESIAN FACE

In [44]:
import sys
import numpy as np
from scipy.io import loadmat
from sklearn import metrics
from sklearn.decomposition import PCA
from sklearn.externals import joblib


#Before training,the mean must be substract
def JointBayesian_Train(trainingset, label, fold = "./"):
    if fold[-1] != '/':
        fold += '/'
    print (trainingset.shape)
    # the total num of image
    n_image = len(label)
    # the dim of features
    n_dim   = trainingset.shape[1]
    # filter the complicate label,for count the total people num
    classes, labels = np.unique(label, return_inverse=True)
    # the total people num
    n_class = len(classes)
    # save each people items
    cur = {}
    withinCount = 0
    # record the count of each people
    numberBuff = np.zeros(n_image)
    maxNumberInOneClass = 0
    for i in range(n_class):
        # get the item of i
        cur[i] = trainingset[labels==i]
        # get the number of the same label persons
        n_same_label = cur[i].shape[0]
        
        if n_same_label > 1:
            withinCount += n_same_label
        if numberBuff[n_same_label] == 0:
            numberBuff[n_same_label] = 1
            maxNumberInOneClass = max(maxNumberInOneClass, n_same_label)
    print ("prepare done, maxNumberInOneClass=", maxNumberInOneClass)

    u  = np.zeros([n_dim, n_class])
    ep = np.zeros([n_dim, withinCount])
    nowp=0
    for i in range(n_class):
        # the mean of cur[i]
        u[:,i] = np.mean(cur[i], 0)
        b = u[:,i].reshape(n_dim, 1)
        n_same_label = cur[i].shape[0]
        if n_same_label > 1:
            ep[:, nowp:nowp+n_same_label] = cur[i].T-b
            nowp += n_same_label

    Su = np.cov(u.T,  rowvar=0)
    Sw = np.cov(ep.T, rowvar=0)
    oldSw = Sw
    SuFG  = {}
    SwG   = {}
    convergence = 1
    min_convergence = 1
    for l in range(2):
        F  = np.linalg.pinv(Sw)
        u  = np.zeros([n_dim, n_class])
        ep = np.zeros([n_dim, n_image])
        nowp = 0
        for mi in range(maxNumberInOneClass + 1):
            if numberBuff[mi] == 1:
		#G = −(mS μ + S ε )−1*Su*Sw−1
                G = -np.dot(np.dot(np.linalg.pinv(mi*Su+Sw), Su), F)
		#Su*(F+mi*G) for u
                SuFG[mi] = np.dot(Su, (F+mi*G))
		#Sw*G for e
                SwG[mi]  = np.dot(Sw, G)
        for i in range(n_class):
            ##print l, i
            nn_class = cur[i].shape[0]
	    #formula 7 in suppl_760
            u[:,i] = np.sum(np.dot(SuFG[nn_class],cur[i].T), 1)
	    #formula 8 in suppl_760
            ep[:,nowp:nowp+nn_class] = cur[i].T+np.sum(np.dot(SwG[nn_class],cur[i].T),1).reshape(n_dim,1)
            nowp = nowp+nn_class

        Su = np.cov(u.T,  rowvar=0)
        Sw = np.cov(ep.T, rowvar=0)
        convergence = np.linalg.norm(Sw-oldSw)/np.linalg.norm(Sw)
        print_info("Iterations-" + str(l) + ": "+ str(convergence))
        if convergence<1e-6:
            print ("Convergence: ", l, convergence)
            break;
        oldSw=Sw

        if convergence < min_convergence:
       	    min_convergence = convergence
            F = np.linalg.pinv(Sw)
            G = -np.dot(np.dot(np.linalg.pinv(2*Su+Sw),Su), F)
            A = np.linalg.pinv(Su+Sw)-(F+G)
            f = open("/content/result/G.pkl","wb")
            pickle.dump(G,f)
            f = open("/content/result/A.pkl","wb")
            pickle.dump(A,f)
            
    F = np.linalg.pinv(Sw)
    G = -np.dot(np.dot(np.linalg.pinv(2*Su+Sw),Su), F)
    A = np.linalg.pinv(Su+Sw) - (F+G)
    f = open("/content/result/G_con.pkl","wb")
    pickle.dump(G,f)
    f = open("/content/result/A_con.pkl","wb")
    pickle.dump(A,f)

    return (A, G)

#ratio of similar,the threshold we always choose in (-1,-2)            
def Verify(A, G, x1, x2):

    x1.shape = (-1,1)
    x2.shape = (-1,1)
    ratio = np.dot(np.dot(np.transpose(x1),A),x1) + np.dot(np.dot(np.transpose(x2),A),x2) - 2*np.dot(np.dot(np.transpose(x1),G),x2)
    return float(ratio)


def PCA_Train(data, result_fold, n_components=2000):
    print_info("PCA training (n_components=%d)..."%n_components)

    pca = PCA(n_components=n_components)
    pca.fit(data)
    joblib.dump(pca, "/content/result/pca_model.m")
     
    print_info("PCA done.")

    return (pca)

def data_pre(data):
    data = np.sqrt(data)
    data = np.divide(data, np.repeat(np.sum(data, 1), data.shape[1]).reshape(data.shape[0], data.shape[1]))
    
    return (data)

def get_ratios(A, G, pair_list):
    ratio = Verify(A, G, pair_list[0], pair_list[1])
    return (ratio)

def excute_performance(dist, y , t_s=-8, t_e=10, t_step=1):
    print ("test size: ", y.shape)
    print ("negative size: ", y[y==0].shape)
    print ("postive size: ",  y[y==1].shape)

    draw_list = []
    while (t_s < t_e):
        pre = dist >= t_s
        y = (y==1)
        report = metrics.classification_report(y_true=y, y_pred=pre)
        print ("threshold: ", t_s)
        print (report)

        t_s += t_step


In [39]:
import pickle
import time
import zipfile
import numpy as np
import random


def get_time_str():
    return (time.strftime("%Y-%m-%d, %H:%M:%S ", time.localtime((time.time()) )))

def print_info(msg):
    print (get_time_str(), msg)


# saving data into pkl
def data_to_pkl(data, file_path):
    print ("Saving data to file(%s). "%(file_path))

    with open(file_path, "w") as f:
        pickle.dump(data,f)
        return (True)

    print ("Occur Error while saving...")
    return (False)

def read_pkl(file_path):
    with open(file_path, "r") as f:
        return pickle.load(f)

def save_draw_file(draw_list):
    st = ""
    for row in draw_list:
        threshold = row[1]
        st += str(threshold) + " "
        for r in row[0]:
            for c in r:
                st += str(c) + " "
        st += "\n"
    with open("draw_file.txt", "w") as f:
        f.write(st)


def report_format(report):
    report = report.split()
    result = np.zeros((3,3))
    result[0][0] = report[5]
    result[0][1] = report[6]
    result[0][2] = report[7]

    result[1][0] = report[10]
    result[1][1] = report[11]
    result[1][2] = report[12]

    result[2][0] = report[17]
    result[2][1] = report[18]
    result[2][2] = report[19]

    return result

if __name__ == "__main__":
    pass

In [ ]:
import shutil
shutil.rmtree("result")

In [40]:
os.makedirs("result")

In [45]:
import sys
import numpy as np
from scipy.io import loadmat
from sklearn import metrics
from sklearn.decomposition import PCA
from sklearn.externals import joblib


def excute_train(train_data=train_images, train_label= train_labels, result_fold="../result/"):

    # data predeal
    data = data_pre(train_data)
    # pca training.
    pca = PCA_Train(data, result_fold)
    data_pca = pca.transform(data)
    filehandler = open("/content/result/pca_wdref.pkl","wb")
    pickle.dump(data_pca, filehandler)
    JointBayesian_Train(data_pca, train_label, result_fold)


if __name__ == "__main__":
    excute_train()

2021-11-05, 05:59:30  PCA training (n_components=2000)...
2021-11-05, 06:01:20  PCA done.
(13232, 2000)
prepare done, maxNumberInOneClass= 530
2021-11-05, 06:11:35  Iterations-0: 0.2870027139831995
2021-11-05, 06:21:21  Iterations-1: 0.09598986343727313


In [46]:
def execute_test(test_data):
    with open("/content/result/A.pkl", "rb") as f:
        A = pickle.load(f)
    with open("/content/result/G.pkl", "rb") as f:
        G = pickle.load(f)
    data  = data_pre(test_data)
    clt_pca = joblib.load("result/pca_model.m")
    data = clt_pca.transform(data)
    #data_to_pkl(data, result_fold+"pca_lfw.pkl")

    #data = read_pkl(result_fold+"pca_lfw.pkl")
    #print (data.shape)

    dist = get_ratios(A, G, data)
    #dist_Extra = get_ratios(A, G, test_Extra, data)

    #dist_all = dist_Intra + dist_Extra
    #dist_all = np.asarray(dist_all)
    return(dist)

In [47]:
outputs= []
for i in range(0,len(X)):
    c= execute_test(X[i])
    outputs.append(c)

In [ ]:
print(outputs)

[-468.10275244872093, -352.77633764899167, -447.0114136181552, -464.23035778986883, -317.5394830017981, -155.07926971133432, -684.4221815913653, -502.4689524514561, -375.12094470538466, -529.055001551785, -358.9278099253472, -432.03637840832727, -334.1122181597153, -372.79762314495514, -354.4356603073932, -207.02502129952705, -286.9189333917996, -240.02294267422158, -198.03162867183494, -368.9266530559667, -363.94142701322517, -860.1939330510786, -383.53519207279663, -210.22574457436411, -547.3773315904479, -914.837118749476, -388.00785362826167, -368.4556902724173, -318.8827847922514, -416.99106038530283, -308.2829069289252, -278.8269624835661, -420.8825799702738, -796.922267211385, -180.24770980536107, -352.870697471804, -176.0014148413742, -500.6205563497938, -327.5175594597607, -234.2548205800052, -785.668616163265, -510.1955339700944, -348.29316635570194, -532.3484679593224, -285.8276079597436, -364.1420715499081, -516.744752193356, -303.2236619134244, -401.650508242539, -489.0748

In [48]:
outputs= np.array(outputs)
Y= np.array(Y)

In [49]:
outputs.shape, Y.shape

((6000,), (6000,))

In [ ]:
excute_performance(outputs, Y, min(outputs), max(outputs), 1)

In [51]:
# Training the Kernel SVM model on the Training set
from sklearn.svm import SVC
classifier = SVC(kernel = 'rbf', random_state = 0)
from sklearn.model_selection import cross_val_score
accuracies = cross_val_score(estimator = classifier, X = outputs.reshape(-1,1), y = Y, cv = 10)
print("Accuracy: {:.2f} %".format(accuracies.mean()*100))
print("Standard Deviation: {:.2f} %".format(accuracies.std()*100))

Accuracy: 89.65 %
Standard Deviation: 1.50 %


In [52]:
from sklearn import metrics
from sklearn.model_selection import GridSearchCV, KFold, StratifiedKFold
from sklearn.metrics import classification_report, accuracy_score, make_scorer
originalclass = []
predictedclass = []
def classification_report_with_accuracy_score(y_true, y_pred):
    originalclass.extend(y_true)
    predictedclass.extend(y_pred)
    return accuracy_score(y_true, y_pred) # return accuracy score

inner_cv = StratifiedKFold(n_splits=10)
outer_cv = StratifiedKFold(n_splits=10)

# Nested CV with parameter optimization
nested_score = cross_val_score(classifier, X=outputs.reshape(-1,1), y=Y, cv=outer_cv, scoring=make_scorer(classification_report_with_accuracy_score))

# Average values in classification report for all folds in a K-fold Cross-validation  
print(classification_report(originalclass, predictedclass)) 

              precision    recall  f1-score   support

           0       0.95      0.84      0.89      3000
           1       0.86      0.95      0.90      3000

    accuracy                           0.90      6000
   macro avg       0.90      0.90      0.90      6000
weighted avg       0.90      0.90      0.90      6000



FISHER FACE


In [30]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
lda = LDA(n_components = 200)
train_images = lda.fit_transform(train_images, train_labels)

In [31]:
outputs=[]
for x in range(0,len(X)):
  c = lda.transform(X[x])
  a= c[0]
  b= c[1]
  dist = np.linalg.norm(a - b)
  outputs.append(dist)

In [32]:
outputs= np.array(outputs)
Y= np.array(Y)

In [33]:
# Training the Kernel SVM model on the Training set
from sklearn.svm import SVC
classifier = SVC(kernel = 'rbf', random_state = 0)
from sklearn.model_selection import cross_val_score
accuracies = cross_val_score(estimator = classifier, X = outputs.reshape(-1,1), y = Y, cv = 10)
print("Accuracy: {:.2f} %".format(accuracies.mean()*100))
print("Standard Deviation: {:.2f} %".format(accuracies.std()*100))

Accuracy: 83.08 %
Standard Deviation: 1.76 %


In [34]:
from sklearn import metrics
from sklearn.model_selection import GridSearchCV, KFold, StratifiedKFold
from sklearn.metrics import classification_report, accuracy_score, make_scorer
originalclass = []
predictedclass = []
def classification_report_with_accuracy_score(y_true, y_pred):
    originalclass.extend(y_true)
    predictedclass.extend(y_pred)
    return accuracy_score(y_true, y_pred) # return accuracy score

inner_cv = StratifiedKFold(n_splits=10)
outer_cv = StratifiedKFold(n_splits=10)

# Nested CV with parameter optimization
nested_score = cross_val_score(classifier, X=outputs.reshape(-1,1), y=Y, cv=outer_cv, scoring=make_scorer(classification_report_with_accuracy_score))

# Average values in classification report for all folds in a K-fold Cross-validation  
print(classification_report(originalclass, predictedclass)) 

              precision    recall  f1-score   support

           0       0.87      0.78      0.82      3000
           1       0.80      0.88      0.84      3000

    accuracy                           0.83      6000
   macro avg       0.83      0.83      0.83      6000
weighted avg       0.83      0.83      0.83      6000

